# SIMULACRO PC2 - ALGORITMOS Y ESTRUCTURA DE DATOS
## Tema: Listas Enlazadas

**Duración:** 2 horas  
**Instrucciones:**
- Resuelve todos los problemas utilizando listas enlazadas
- Se valorará la eficiencia algorítmica y la claridad del código
- **IMPORTANTE:** El uso de IA (ChatGPT, Copilot, etc.) será penalizado con 0 puntos en el problema detectado
- Justifica tus soluciones con comentarios cuando sea necesario

---

## CASO DE ESTUDIO: SISTEMA LOGÍSTICO DEL PUERTO DE CHANCAY

El Puerto de Chancay, megapuerto que conecta Perú con Asia, necesita un sistema de gestión logística para optimizar el manejo de contenedores y buques. Tu tarea es implementar diferentes módulos utilizando listas enlazadas.

### Contexto Técnico:
- Cada contenedor tiene: código único, peso (toneladas), origen, destino, prioridad (1=urgente, 2=normal, 3=diferible)
- Los buques llegan en diferentes horarios y tienen capacidad limitada
- El sistema debe manejar colas de espera, prioridades y optimización de rutas

---

In [ ]:
# CLASES BASE - NO MODIFICAR
class Contenedor:
    def __init__(self, codigo, peso, origen, destino, prioridad=2):
        self.codigo = codigo
        self.peso = peso
        self.origen = origen
        self.destino = destino
        self.prioridad = prioridad  # 1=urgente, 2=normal, 3=diferible
    
    def __str__(self):
        return f"[{self.codigo}] {self.peso}T ({self.origen}→{self.destino}) P{self.prioridad}"

class Nodo:
    def __init__(self, contenedor):
        self.contenedor = contenedor
        self.siguiente = None

---
## PROBLEMA 1: SISTEMA DE COLA PRIORITARIA DE CONTENEDORES (4 puntos)

Implementa una lista enlazada que maneje una cola prioritaria de contenedores esperando ser cargados. Los contenedores urgentes (prioridad 1) deben ir al inicio, los normales (prioridad 2) en el medio, y los diferibles (prioridad 3) al final.

**Requerimientos:**
1. `agregar_contenedor(contenedor)`: Inserta el contenedor según su prioridad
2. `cargar_siguiente()`: Retorna y elimina el contenedor de mayor prioridad
3. `mostrar_cola()`: Muestra todos los contenedores en orden de prioridad
4. `contar_por_prioridad(prioridad)`: Cuenta contenedores de una prioridad específica

**Caso de prueba:**
```python
# Contenedores llegando al puerto
c1 = Contenedor("CHN001", 25.5, "Shanghai", "Lima", 2)
c2 = Contenedor("URG001", 15.0, "Lima", "Paita", 1)  # Urgente
c3 = Contenedor("DIF001", 30.0, "Callao", "Iquitos", 3)  # Diferible
```

In [12]:
class ColaPrioritaria:
    def __init__(self):
        self.cabeza = None

    def agregar_contenedor(self, contenedor):
        nuevo_nodo = Nodo(contenedor)
        if self.cabeza is None:
            self.cabeza = nuevo_nodo
            return

        if contenedor.prioridad == 1:  # urgentes al inicio
            if self.cabeza.contenedor.prioridad != 1:
                nuevo_nodo.siguiente = self.cabeza
                self.cabeza = nuevo_nodo
            else:
                actual = self.cabeza
                while actual.siguiente and actual.siguiente.contenedor.prioridad == 1:
                    actual = actual.siguiente
                nuevo_nodo.siguiente = actual.siguiente
                actual.siguiente = nuevo_nodo
            return

        if contenedor.prioridad == 3:  # diferibles al final
            actual = self.cabeza
            while actual.siguiente:
                actual = actual.siguiente
            actual.siguiente = nuevo_nodo
            return

        if contenedor.prioridad == 2:  # normales en medio
            if self.cabeza.contenedor.prioridad > 2:
                nuevo_nodo.siguiente = self.cabeza
                self.cabeza = nuevo_nodo
                return
            
            actual = self.cabeza
            while actual.siguiente and actual.siguiente.contenedor.prioridad <= 2:
                actual = actual.siguiente
            nuevo_nodo.siguiente = actual.siguiente
            actual.siguiente = nuevo_nodo

    def cargar_siguiente(self):
        if not self.cabeza:
            return None
        contenedor_cargado = self.cabeza.contenedor
        self.cabeza = self.cabeza.siguiente
        return contenedor_cargado

    def mostrar_cola(self):
        actual = self.cabeza
        while actual:
            print(actual.contenedor)
            actual = actual.siguiente

    def contar_por_prioridad(self, prioridad):
        contador = 0
        actual = self.cabeza
        while actual:
            if actual.contenedor.prioridad == prioridad:
                contador += 1
            actual = actual.siguiente
        return contador

# CÓDIGO DE PRUEBA
cola = ColaPrioritaria()
# Agregar tus casos de prueba aquí
cola.agregar_contenedor(Contenedor("C1", 10, "Lima", "Tokio", 2))
cola.agregar_contenedor(Contenedor("C2", 5, "Lima", "NY", 1))
cola.agregar_contenedor(Contenedor("C3", 8, "Lima", "Santos", 3))
cola.agregar_contenedor(Contenedor("C4", 12, "Lima", "Hamburgo", 1))
cola.mostrar_cola()
print("Contar urgentes:", cola.contar_por_prioridad(1))
print("Cargando siguiente:", cola.cargar_siguiente())
cola.mostrar_cola()

[C2] 5T (Lima→NY) P1
[C4] 12T (Lima→Hamburgo) P1
[C1] 10T (Lima→Tokio) P2
[C3] 8T (Lima→Santos) P3
Contar urgentes: 2
Cargando siguiente: [C2] 5T (Lima→NY) P1
[C4] 12T (Lima→Hamburgo) P1
[C1] 10T (Lima→Tokio) P2
[C3] 8T (Lima→Santos) P3


---
## PROBLEMA 2: OPTIMIZADOR DE CARGA POR PESO (4 puntos)

Los buques tienen capacidad máxima de peso. Implementa un sistema que encuentre la mejor combinación de contenedores que no exceda la capacidad del buque, maximizando la cantidad de contenedores transportados.

**Requerimientos:**
1. `cargar_buque(lista_contenedores, capacidad_maxima)`: Retorna lista enlazada con contenedores seleccionados
2. `peso_total()`: Calcula el peso total de contenedores en la lista
3. `optimizar_por_destino(destino)`: Prioriza contenedores con el mismo destino
4. Debe intentar incluir la mayor cantidad de contenedores sin exceder la capacidad

**Caso de prueba:**
- Buque con capacidad: 80 toneladas
- Contenedores disponibles: [25.5T, 30T, 15T, 35T, 20T, 10T]
- El sistema debe seleccionar la mejor combinación

In [11]:
class Nodo:
    def __init__(self, contenedor):
        self.contenedor = contenedor
        self.siguiente = None

class OptimizadorCarga:
    def __init__(self):
        self.cabeza = None

    def cargar_buque(self, lista_contenedores, capacidad_maxima):
        lista_contenedores.sort(key=lambda c: c.peso)
        peso_actual = 0
        for contenedor in lista_contenedores:
            if peso_actual + contenedor.peso <= capacidad_maxima:
                self.agregar_contenedor(contenedor)
                peso_actual += contenedor.peso

    def agregar_contenedor(self, contenedor):
        nuevo_nodo = Nodo(contenedor)
        if not self.cabeza:
            self.cabeza = nuevo_nodo
        else:
            actual = self.cabeza
            while actual.siguiente:
                actual = actual.siguiente
            actual.siguiente = nuevo_nodo

    def peso_total(self):
        total = 0
        actual = self.cabeza
        while actual:
            total += actual.contenedor.peso
            actual = actual.siguiente
        return total

    def optimizar_por_destino(self, destino):
        contenedores_destino = []
        otros_contenedores = []
        actual = self.cabeza
        while actual:
            if actual.contenedor.destino == destino:
                contenedores_destino.append(actual.contenedor)
            else:
                otros_contenedores.append(actual.contenedor)
            actual = actual.siguiente
        self.cabeza = None
        for c in contenedores_destino + otros_contenedores:
            self.agregar_contenedor(c)

    def mostrar_carga(self):
        actual = self.cabeza
        while actual:
            print(actual.contenedor)
            actual = actual.siguiente


# CÓDIGO DE PRUEBA
opt = OptimizadorCarga()
# Crear lista de contenedores de prueba
contenedores = [
    Contenedor("X1", 7, "Lima", "NY"),
    Contenedor("X2", 3, "Lima", "NY"),
    Contenedor("X3", 10, "Lima", "Tokio"),
    Contenedor("X4", 5, "Lima", "NY"),
]
opt.cargar_buque(contenedores, 15)
opt.mostrar_carga()
print("Peso total:", opt.peso_total())
opt.optimizar_por_destino("NY")
print("Optimizado por destino NY:")
opt.mostrar_carga()

[X2] 3T (Lima→NY) P2
[X4] 5T (Lima→NY) P2
[X1] 7T (Lima→NY) P2
Peso total: 15
Optimizado por destino NY:
[X2] 3T (Lima→NY) P2
[X4] 5T (Lima→NY) P2
[X1] 7T (Lima→NY) P2


---
## PROBLEMA 3: SISTEMA DE RASTREO DE RUTAS MARÍTIMAS (4 puntos)

Implementa un sistema que rastree la ruta de un contenedor desde su origen hasta su destino, pasando por diferentes puertos intermedios. La ruta se representa como una lista enlazada de puertos.

**Requerimientos:**
1. `agregar_puerto(nombre_puerto, tiempo_llegada)`: Añade puerto a la ruta
2. `buscar_puerto(nombre_puerto)`: Verifica si un puerto está en la ruta
3. `tiempo_total_viaje()`: Calcula duración total del viaje
4. `eliminar_escala(puerto)`: Elimina un puerto intermedio de la ruta
5. `ruta_alternativa(puerto_problematico, puerto_alternativo)`: Reemplaza un puerto

**Caso de prueba:**
- Ruta: Shanghai → Singapur → Chancay → Callao → Paita
- Tiempos: 0 → 72h → 168h → 180h → 200h
- Simular problema en Singapur y usar ruta alternativa por Hong Kong

In [9]:
class Puerto:
    def __init__(self, nombre, tiempo_llegada):
        self.nombre = nombre
        self.tiempo_llegada = tiempo_llegada
    def __str__(self):
        return f"{self.nombre} (T+{self.tiempo_llegada}h)"

class NodoPuerto:
    def __init__(self, puerto):
        self.puerto = puerto
        self.siguiente = None

class RastreadorRutas:
    def __init__(self, contenedor_codigo):
        self.contenedor_codigo = contenedor_codigo
        self.cabeza = None

    def agregar_puerto(self, nombre_puerto, tiempo_llegada):
        nuevo_puerto = Puerto(nombre_puerto, tiempo_llegada)
        nuevo_nodo = NodoPuerto(nuevo_puerto)
        if not self.cabeza or self.cabeza.puerto.tiempo_llegada > tiempo_llegada:
            nuevo_nodo.siguiente = self.cabeza
            self.cabeza = nuevo_nodo
            return
        actual = self.cabeza
        while actual.siguiente and actual.siguiente.puerto.tiempo_llegada < tiempo_llegada:
            actual = actual.siguiente
        nuevo_nodo.siguiente = actual.siguiente
        actual.siguiente = nuevo_nodo

    def buscar_puerto(self, nombre_puerto):
        actual = self.cabeza
        while actual:
            if actual.puerto.nombre == nombre_puerto:
                return True
            actual = actual.siguiente
        return False

    def tiempo_total_viaje(self):
        if not self.cabeza:
            return 0
        actual = self.cabeza
        while actual.siguiente:
            actual = actual.siguiente
        return actual.puerto.tiempo_llegada

    def eliminar_escala(self, puerto_nombre):
        if not self.cabeza:
            return
        if self.cabeza.puerto.nombre == puerto_nombre:
            self.cabeza = self.cabeza.siguiente
            return
        actual = self.cabeza
        while actual.siguiente and actual.siguiente.puerto.nombre != puerto_nombre:
            actual = actual.siguiente
        if actual.siguiente:
            actual.siguiente = actual.siguiente.siguiente

    def ruta_alternativa(self, puerto_problematico, puerto_alternativo, nuevo_tiempo):
        self.eliminar_escala(puerto_problematico)
        self.agregar_puerto(puerto_alternativo, nuevo_tiempo)

    def mostrar_ruta(self):
        ruta = []
        actual = self.cabeza
        while actual:
            ruta.append(str(actual.puerto))
            actual = actual.siguiente
        print(" → ".join(ruta))
# CÓDIGO DE PRUEBA
rastreador = RastreadorRutas("CHN001")
rastreador.agregar_puerto("Lima", 0)
rastreador.agregar_puerto("Panamá", 24)
rastreador.agregar_puerto("NY", 72)
rastreador.mostrar_ruta()
# Construir la ruta de ejemplo
print("Buscar Panamá:", rastreador.buscar_puerto("Panamá"))
print("Tiempo total viaje:", rastreador.tiempo_total_viaje())
rastreador.eliminar_escala("Panamá")
rastreador.mostrar_ruta()
rastreador.ruta_alternativa("NY", "Rotterdam", 60)
rastreador.mostrar_ruta()

Lima (T+0h) → Panamá (T+24h) → NY (T+72h)
Buscar Panamá: True
Tiempo total viaje: 72
Lima (T+0h) → NY (T+72h)
Lima (T+0h) → Rotterdam (T+60h)


---
## PROBLEMA 4: DETECTOR DE CONTENEDORES DUPLICADOS (4 puntos)

En el caos del puerto, a veces se registran contenedores duplicados. Implementa un sistema que detecte y elimine registros duplicados de contenedores, manteniendo solo el registro más reciente.

**Requerimientos:**
1. `agregar_registro(contenedor, timestamp)`: Añade registro con marca de tiempo
2. `detectar_duplicados()`: Encuentra contenedores con mismo código
3. `eliminar_duplicados()`: Elimina registros duplicados, mantiene el más reciente
4. `generar_reporte()`: Muestra estadísticas de duplicados encontrados
5. **Optimización:** El algoritmo debe ser eficiente O(n) en una sola pasada

**Caso de prueba:**
- Registros con diferentes timestamps del mismo contenedor "CHN001"
- Contenedores únicos que no deben eliminarse
- Verificar que se mantiene solo el registro más reciente

In [8]:
import time

class Contenedor:
    def __init__(self, codigo, peso, origen, destino, prioridad=2):
        self.codigo = codigo
        self.peso = peso
        self.origen = origen
        self.destino = destino
        self.prioridad = prioridad  # 1=urgente, 2=normal, 3=diferible

    def __str__(self):
        return f"[{self.codigo}] {self.peso}T ({self.origen}→{self.destino}) P{self.prioridad}"

class RegistroContenedor:
    def __init__(self, contenedor, timestamp):
        self.contenedor = contenedor
        self.timestamp = timestamp
    def __str__(self):
        return f"{self.contenedor} - {self.timestamp}"

class NodoRegistro:
    def __init__(self, registro):
        self.registro = registro
        self.siguiente = None

class DetectorDuplicados:
    def __init__(self):
        self.cabeza = None
        self.duplicados_encontrados = 0

    def agregar_registro(self, contenedor, timestamp=None):
        if timestamp is None:
            timestamp = time.time()
        registro = RegistroContenedor(contenedor, timestamp)
        nuevo_nodo = NodoRegistro(registro)
        nuevo_nodo.siguiente = self.cabeza
        self.cabeza = nuevo_nodo

    def detectar_duplicados(self):
        codigos = {}
        duplicados = []
        actual = self.cabeza
        while actual:
            codigo = actual.registro.contenedor.codigo
            if codigo in codigos:
                if codigo not in duplicados:
                    duplicados.append(codigo)
            else:
                codigos[codigo] = True
            actual = actual.siguiente
        return duplicados

    def eliminar_duplicados(self):
        if not self.cabeza:
            return
        vistos = {}
        actual = self.cabeza
        previo = None
        while actual:
            codigo = actual.registro.contenedor.codigo
            if codigo in vistos:
                if actual.registro.timestamp < vistos[codigo].registro.timestamp:
                    previo.siguiente = actual.siguiente
                    self.duplicados_encontrados += 1
                else:
                    vistos[codigo] = actual
                    previo = actual
            else:
                vistos[codigo] = actual
                previo = actual
            actual = actual.siguiente

    def generar_reporte(self):
        print(f"Se encontraron y eliminaron {self.duplicados_encontrados} registros duplicados.")

    def mostrar_registros(self):
        actual = self.cabeza
        while actual:
            print(actual.registro)
            actual = actual.siguiente

# CÓDIGO DE PRUEBA
det = DetectorDuplicados()
c1 = Contenedor("D1", 5, "Lima", "NY")
c2 = Contenedor("D2", 7, "Lima", "Tokio")
det.agregar_registro(c1, 100)
det.agregar_registro(c2, 200)
det.agregar_registro(c1, 300)  # duplicado
det.mostrar_registros()
print("Duplicados detectados:", det.detectar_duplicados())
det.eliminar_duplicados()
det.generar_reporte()
det.mostrar_registros()


[D1] 5T (Lima→NY) P2 - 300
[D2] 7T (Lima→Tokio) P2 - 200
[D1] 5T (Lima→NY) P2 - 100
Duplicados detectados: ['D1']
Se encontraron y eliminaron 1 registros duplicados.
[D1] 5T (Lima→NY) P2 - 300
[D2] 7T (Lima→Tokio) P2 - 200


---
## PROBLEMA 5: SIMULADOR DE CONVOY DE BUQUES (4 puntos)

El Puerto de Chancay debe coordinar la llegada de múltiples buques que viajan en convoy. Implementa un simulador que maneje la formación, navegación y llegada de buques, considerando que algunos pueden salir del convoy o nuevos pueden unirse en el camino.

**Requerimientos:**
1. `formar_convoy(lista_buques)`: Organiza buques en orden de llegada planificada
2. `buque_sale_convoy(nombre_buque)`: Remueve buque que abandona el convoy
3. `buque_se_une(buque, posicion)`: Inserta nuevo buque en posición específica
4. `reorganizar_por_emergencia(buque_emergencia)`: Mueve buque con emergencia al frente
5. `simular_llegada()`: Simula llegada secuencial al puerto
6. **Bonus:** `calcular_tiempo_atraque_total()`: Estima tiempo total considerando que cada buque toma 4 horas en atracar

**Caso de prueba:**
- Convoy inicial: ["COSCO_Shanghai", "Maersk_Europe", "MSC_Asia", "Hapag_Germany"]
- "Maersk_Europe" abandona convoy
- "Emergency_Vessel" se une al frente por emergencia médica
- Simular proceso completo de llegada al Puerto de Chancay

In [1]:
class Buque:
    def __init__(self, nombre, capacidad_contenedores, eta_original):
        self.nombre = nombre
        self.capacidad = capacidad_contenedores
        self.eta_original = eta_original
    def __str__(self):
        return f"{self.nombre} (Cap:{self.capacidad}, ETA:{self.eta_original})"

class NodoBuque:
    def __init__(self, buque):
        self.buque = buque
        self.siguiente = None

class SimuladorConvoy:
    def __init__(self):
        self.cabeza = None
        self.tiempo_atraque_por_buque = 4

    def formar_convoy(self, lista_buques):
        lista_buques.sort(key=lambda b: b.eta_original)
        for buque in lista_buques:
            self.buque_se_une(buque)

    def buque_sale_convoy(self, nombre_buque):
        if not self.cabeza:
            return
        if self.cabeza.buque.nombre == nombre_buque:
            self.cabeza = self.cabeza.siguiente
            return
        actual = self.cabeza
        while actual.siguiente and actual.siguiente.buque.nombre != nombre_buque:
            actual = actual.siguiente
        if actual.siguiente:
            actual.siguiente = actual.siguiente.siguiente

    def buque_se_une(self, buque, posicion=None):
        nuevo_nodo = NodoBuque(buque)
        if posicion == 0 or not self.cabeza:
            nuevo_nodo.siguiente = self.cabeza
            self.cabeza = nuevo_nodo
            return
        if posicion is None:
            actual = self.cabeza
            while actual.siguiente:
                actual = actual.siguiente
            actual.siguiente = nuevo_nodo
            return
        actual = self.cabeza
        contador = 0
        while actual.siguiente and contador < posicion - 1:
            actual = actual.siguiente
            contador += 1
        nuevo_nodo.siguiente = actual.siguiente
        actual.siguiente = nuevo_nodo

    def reorganizar_por_emergencia(self, nombre_buque_emergencia):
        if not self.cabeza or self.cabeza.buque.nombre == nombre_buque_emergencia:
            return
        actual = self.cabeza
        previo = None
        while actual and actual.buque.nombre != nombre_buque_emergencia:
            previo = actual
            actual = actual.siguiente
        if actual:
            previo.siguiente = actual.siguiente
            actual.siguiente = self.cabeza
            self.cabeza = actual

    def simular_llegada(self):
        tiempo_actual = 0
        actual = self.cabeza
        while actual:
            print(f"T+{tiempo_actual}h: Llega y atraca el buque {actual.buque.nombre}")
            tiempo_actual += self.tiempo_atraque_por_buque
            actual = actual.siguiente

    def calcular_tiempo_atraque_total(self):
        num_buques = 0
        actual = self.cabeza
        while actual:
            num_buques += 1
            actual = actual.siguiente
        return num_buques * self.tiempo_atraque_por_buque

    def mostrar_convoy(self):
        convoy = []
        actual = self.cabeza
        while actual:
            convoy.append(actual.buque.nombre)
            actual = actual.siguiente
        print("Convoy actual: " + " → ".join(convoy))


# CÓDIGO DE PRUEBA
convoy = SimuladorConvoy()
b1 = Buque("Evergreen", 200, 10)
b2 = Buque("Maersk", 150, 8)
b3 = Buque("Hanjin", 100, 12)
convoy.formar_convoy([b1, b2, b3])
convoy.mostrar_convoy()
convoy.simular_llegada()
print("Tiempo total atraque:", convoy.calcular_tiempo_atraque_total())
convoy.reorganizar_por_emergencia("Hanjin")
convoy.mostrar_convoy()
convoy.buque_sale_convoy("Evergreen")
convoy.mostrar_convoy()

Convoy actual: Maersk → Evergreen → Hanjin
T+0h: Llega y atraca el buque Maersk
T+4h: Llega y atraca el buque Evergreen
T+8h: Llega y atraca el buque Hanjin
Tiempo total atraque: 12
Convoy actual: Hanjin → Maersk → Evergreen
Convoy actual: Hanjin → Maersk


---
## CRITERIOS DE EVALUACIÓN

### Distribución de Puntos (Total: 20 puntos)
- **Problema 1:** 4 puntos (Cola prioritaria)
- **Problema 2:** 4 puntos (Optimización de carga)
- **Problema 3:** 4 puntos (Rastreo de rutas)
- **Problema 4:** 4 puntos (Detector de duplicados)
- **Problema 5:** 4 puntos (Simulador de convoy)

### Rubrica por problema:
- **4 puntos:** Implementación completa y correcta, maneja todos los casos edge, código optimizado
- **3 puntos:** Implementación correcta con casos básicos, funcionalidad principal completa
- **2 puntos:** Implementación parcial, algunas funciones trabajando correctamente
- **1 punto:** Intento de implementación, estructura básica presente
- **0 puntos:** No implementado, uso detectado de IA, o solución incorrecta

### Aspectos Evaluados:
1. **Corrección Algorítmica (40%):** La solución resuelve el problema planteado
2. **Eficiencia (25%):** Complejidad temporal y espacial apropiada
3. **Manejo de Casos Edge (20%):** Lista vacía, un elemento, elementos duplicados, etc.
4. **Calidad de Código (15%):** Claridad, comentarios apropiados, nombres descriptivos

### ⚠️ IMPORTANTE - POLÍTICA ANTI-IA:
- El uso de herramientas de IA será detectado mediante análisis de patrones de código
- Cualquier problema donde se detecte uso de IA recibirá **0 puntos**
- Se valorará la originalidad y el estilo personal de programación
- Las explicaciones deben ser propias y demostrar comprensión real

**Buena suerte**